<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


## OpenAI Assistant with Code Interpreter

- Executed in colab

Let's work through another example of creating an assistant with the code-interpreter tool!

In [1]:
from openai import OpenAI
import os
from google.colab import userdata


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI()

### STEP 1: ASSISTANT CREATION

In [3]:
assistant = client.beta.assistants.create(
    name="Mortgage Bot",
    instructions="You use Python code to help answer questions about mortgage and interest payments.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo"
)

### STEP 2: THREAD CREATION

In [4]:
thread = client.beta.threads.create()

### STEP 3: ADD MESSAGE TO THREAD

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to buy a house that costs $2.1 Milion on a 30-yr fixed loan at 7.8% interest. What will my monthly payments be?"
)

### STEP 4: RUN THREAD WITH ASSISTANT

In [10]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Give detailed analysis and considerations for the user question."
)

In [11]:
run.status

'queued'

In [12]:
import time

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )

    print(run.status)

    time.sleep(1)

completed


In [ ]:
# It shows different status like, queued, In progress, Completed

### STEP 5: DISPLAY RESPONSE

In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages

SyncCursorPage[Message](data=[Message(id='msg_8JVzXakIWeEFUK1EyD28veB6', assistant_id='asst_rvcP3MupeSdKqeLB80IPGX41', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please note that this is an estimate and the actual monthly payment may vary slightly depending on the exact terms of the loan and any additional fees or taxes.'), type='text')], created_at=1735951812, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_hWgnswbrDcamlr5S3Jzwk8yD', status=None, thread_id='thread_4HfR4M8nT3gKGgov6KQ3bXFr'), Message(id='msg_hIHLa8I4jKnhui38PGzYNX3r', assistant_id='asst_rvcP3MupeSdKqeLB80IPGX41', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The monthly mortgage payment for a $2.1 million house on a 30-year fixed loan at 7.8% interest would be approximately $15,117.28.\n\nTherefore, if you take out a 30-year fixed loan at 7.8% interest to bu

In [14]:
for thread_message in messages.data[::-1]:
    print(thread_message.content[0].text.value)
    print('\n')

I want to buy a house that costs $2.1 Milion on a 30-yr fixed loan at 7.8% interest. What will my monthly payments be?


To calculate the monthly mortgage payments for a $2.1 million house on a 30-year fixed loan at 7.8% interest, we can use the formula for calculating the monthly payment on a fixed-rate mortgage.

The formula is:

\[ M = P \times \frac{r(1+r)^n}{(1+r)^n - 1} \]

where:
- \( M \) is the monthly payment
- \( P \) is the principal loan amount (the cost of the house)
- \( r \) is the monthly interest rate (annual interest rate divided by 12 months)
- \( n \) is the total number of payments (loan term in years multiplied by 12 months)

Given:
- Principal amount (\( P \)): $2.1 million
- Annual interest rate: 7.8%
- Loan term: 30 years

We first need to calculate the monthly interest rate (\( r \)) by dividing the annual interest rate by 12. Then we can calculate the monthly payment using the formula mentioned above. Let's calculate that.


The monthly mortgage payment for 

In [15]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

### STEP 6: CONTINUE CONVERSATION

In [16]:
def wait_on_run(run, thread):
    '''
    Simple function that waits on the run of a particular thread.
    Returns the run once the thread has been run.
    '''
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [17]:
client.beta.threads.messages.list(thread_id=thread.id)

SyncCursorPage[Message](data=[Message(id='msg_8JVzXakIWeEFUK1EyD28veB6', assistant_id='asst_rvcP3MupeSdKqeLB80IPGX41', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please note that this is an estimate and the actual monthly payment may vary slightly depending on the exact terms of the loan and any additional fees or taxes.'), type='text')], created_at=1735951812, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_hWgnswbrDcamlr5S3Jzwk8yD', status=None, thread_id='thread_4HfR4M8nT3gKGgov6KQ3bXFr'), Message(id='msg_hIHLa8I4jKnhui38PGzYNX3r', assistant_id='asst_rvcP3MupeSdKqeLB80IPGX41', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The monthly mortgage payment for a $2.1 million house on a 30-year fixed loan at 7.8% interest would be approximately $15,117.28.\n\nTherefore, if you take out a 30-year fixed loan at 7.8% interest to bu

#### Add message to existing Thread

In [18]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="What if I put a down payment of $200k on the house, how would that change the monthly payments?"
)

In [19]:
message

Message(id='msg_B4GMX90pFnSJbwUJ9hDD1I3f', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What if I put a down payment of $200k on the house, how would that change the monthly payments?'), type='text')], created_at=1735951948, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_4HfR4M8nT3gKGgov6KQ3bXFr')

In [20]:
# Execute our run
#Create run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
# WE CAN REQUEST IN 'ASC' ORDER
# WE CAN ALSO REQUEST ONLY MESSAGES AFTER A PARTICULAR ID!
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id  # Here  after=message.id will help us to get only latest asked message Q next answer only. Not all history
)

display_thread_messages(messages)

If you make a down payment of $200,000 towards the $2.1 million house, the loan amount (principal) will be reduced by the amount of the down payment. The new principal will be $2.1 million - $200,000 = $1.9 million.

We can now recalculate the monthly mortgage payment using the new principal amount of $1.9 million. Let's calculate the new monthly payment.


With a down payment of $200,000 towards the $2.1 million house, reducing the principal to $1.9 million, the new monthly mortgage payment would be approximately $13,677.54.

Therefore, by making a down payment of $200,000, your monthly mortgage payment would decrease to around $13,677.54.




-----

### Optional: Review the Run Steps the program took to get the reply

In [21]:
client.beta.threads.runs.list(thread.id)

SyncCursorPage[Run](data=[Run(id='run_U7VmKnt1ZBEiArLDTY1VfpHo', assistant_id='asst_rvcP3MupeSdKqeLB80IPGX41', cancelled_at=None, completed_at=1735952063, created_at=1735952058, expires_at=None, failed_at=None, incomplete_details=None, instructions='You use Python code to help answer questions about mortgage and interest payments.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1735952059, status='completed', thread_id='thread_4HfR4M8nT3gKGgov6KQ3bXFr', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=248, prompt_tokens=2303, total_tokens=2551, prompt_token_details={'cached_tokens': 0}), temperature=1.0, top_p=1.0, tool_resources={}), Run(id='run_hWgnswbrDcamlr5S3Jzwk8yD', assistant_id='asst_rvcP3Mup

In [22]:
#Run steps
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [23]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ALBZPNwLBmrxA8UpAt0xdC6T'), type='message_creation')


ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_9g2nkW60zanLDYAsX2FedJzY', code_interpreter=CodeInterpreter(input='# New principal amount after down payment\r\nnew_principal = principal - 200000  # $1.9 million\r\n\r\n# Calculate monthly payment with the new principal amount\r\nnew_monthly_payment = new_principal * (monthly_interest_rate * (1 + monthly_interest_rate) ** total_num_payments) / ((1 + monthly_interest_rate) ** total_num_payments - 1)\r\nnew_monthly_payment', outputs=[]), type='code_interpreter')], type='tool_calls')


MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_mFsjjxjTNmD6jm0iO0GrKgVV'), type='message_creation')




### Optional: Delete Assistant

In [24]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_rvcP3MupeSdKqeLB80IPGX41', created_at=1735951748, description=None, instructions='You use Python code to help answer questions about mortgage and interest payments.', metadata={}, model='gpt-3.5-turbo', name='Mortgage Bot', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0), Assistant(id='asst_QRRT3wpFSVyX0noIyIrI94ve', created_at=1735951712, description=None, instructions='You use Python code to help answer questions about mortgage and interest payments.', metadata={}, model='gpt-3.5-turbo', name='Mortgage Bot', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)]


In [25]:
my_assistants.data[0].id

'asst_rvcP3MupeSdKqeLB80IPGX41'

In [26]:
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_rvcP3MupeSdKqeLB80IPGX41', deleted=True, object='assistant.deleted')
